In [3]:

import keras

Using TensorFlow backend.


In [0]:
import keras
import tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Churn_Modelling.csv to Churn_Modelling.csv


In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['Churn_Modelling.csv']))
#df2 = pd.read_csv(io.BytesIO(uploaded['Churn_modelling.csv']))

In [6]:
df.head(4)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0


In [7]:
X = df.iloc[:, 3:13]
X
y = df.iloc[:, -1]
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [0]:
geography = pd.get_dummies(df['Geography'], drop_first=True)
gender = pd.get_dummies(df['Gender'], drop_first=True)


In [9]:
X = pd.concat([X, geography, gender], axis = 1)
X.drop(['Geography', 'Gender'], axis = 1, inplace = True)
X

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [0]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [64]:
X_train

array([[ 0.16958176, -0.46460796,  0.00666099, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-2.30455945,  0.30102557, -1.37744033, ...,  1.75486502,
        -0.57369368,  0.91601335],
       [-1.19119591, -0.94312892, -1.031415  , ..., -0.5698444 ,
        -0.57369368, -1.09168714],
       ...,
       [ 0.9015152 , -0.36890377,  0.00666099, ..., -0.5698444 ,
        -0.57369368,  0.91601335],
       [-0.62420521, -0.08179119,  1.39076231, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-0.28401079,  0.87525072, -1.37744033, ...,  1.75486502,
        -0.57369368, -1.09168714]])

In [0]:
from keras.models import Sequential # Sequential is used to make ANN, CNN type neural network.
from keras.layers import Dense, Embedding, Flatten, LeakyReLU, BatchNormalization   # Dense is used to create hidden layers in NN
from keras.layers import Dropout  # Dropout regularisation technique
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.activations import relu, sigmoid


In [0]:
#We have made a model that can fit into kerasclassifier for setting best hyperparameter
def create_model(layers, activations):
    model = Sequential()
    for i, (nodes) in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activations))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activations))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    


In [127]:
 # Create model can be used directly to make a model of required units of neurons and activations functions
 j = create_model((15,15,10), relu)

 j.fit(X_train, y_train, epochs=10)

Epoch 1/10
8000/8000 [==============================] - 1s 125us/step - loss: 0.5719 - accuracy: 0.7365
Epoch 2/10
8000/8000 [==============================] - 1s 105us/step - loss: 0.4965 - accuracy: 0.7952
Epoch 3/10
8000/8000 [==============================] - 1s 103us/step - loss: 0.4848 - accuracy: 0.8002
Epoch 4/10
8000/8000 [==============================] - 1s 106us/step - loss: 0.4611 - accuracy: 0.8089
Epoch 5/10
8000/8000 [==============================] - 1s 104us/step - loss: 0.4571 - accuracy: 0.8089
Epoch 6/10
8000/8000 [==============================] - 1s 103us/step - loss: 0.4454 - accuracy: 0.8134
Epoch 7/10
8000/8000 [==============================] - 1s 103us/step - loss: 0.4379 - accuracy: 0.8189
Epoch 8/10
8000/8000 [==============================] - 1s 104us/step - loss: 0.4248 - accuracy: 0.8195
Epoch 9/10
8000/8000 [==============================] - 1s 101us/step - loss: 0.4254 - accuracy: 0.8217
Epoch 10/10
8000/8000 [==============================] - 1s 105u

In [0]:
model = KerasClassifier(build_fn=create_model, verbose=0)


layer = [(20,), (40, 20), (45, 30, 15)]
activate = ['sigmoid', 'relu']
param_grid = dict(layers=layer, activations=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)



In [0]:
grid_result = grid.fit(X_train, y_train)



In [107]:
 [grid_result.best_score_,grid_result.best_params_]

[0.8582500100135804,
 {'activations': 'relu',
  'batch_size': 128,
  'epochs': 30,
  'layers': (45, 30, 15)}]

In [0]:
#predicing on test set
y_pred = grid.predict(X_test)

y_pred = (y_pred >0.5)


In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

from sklearn.metrics import accuracy_score

score = accuracy_score(y_test, y_pred)

In [110]:
score

0.864